# Prep

In [13]:
import pandas as pd
import numpy as np
import pprint
import re

## Functions

### Extract

In [2]:
def get_raw_workouts(fp):
  all_workouts = []
  counter = 0

  with open(fp, encoding='utf-8') as f:
    clean_strings = [line.strip() for line in f.readlines()]
    no_empty_lines = list(filter(None, clean_strings))
    no_empty_lines.pop(0) # tirar a primeira linha, que é vazia

    single_workout = []
    for line in no_empty_lines:
      if 'Week' in line:
        if counter > 0:
          all_workouts.append(single_workout)
          single_workout = []
        counter += 1
      single_workout.append(line)

  return all_workouts

### Transform

In [54]:
def get_workout_dict(raw_workout_list):
    workout_listdict = []

    for workout in raw_workout_list:
        workout_description = workout[0].split(';')
        description_info = [item.strip() for item in workout_description[0].strip('"').split('·')]

        workout_dict = {
            #'raw_description': f'{workout[0]}', #só pra conferir por enquanto, retirar depois
            'description': {
                'day_name': description_info[0],
                'day_number': description_info[1],
                'week_number': description_info[2],
                'plan_name': description_info[3],
                'plan_function': description_info[4],
                'start_time': workout_description[1].strip('"'),
                'duration': workout_description[2].strip('"')
            },
            'exercises': []
        }

        current_exercise = {'exercise': '', 'data': []}
        for line in workout[1:]:
            if len(line) > 13:
                if len(current_exercise['exercise']) > 0:
                    workout_dict['exercises'].append(current_exercise)
                current_exercise = {'exercise': line, 'data': []}
            else:
                current_exercise['data'].append(line)
        workout_dict['exercises'].append(current_exercise) # necessary to append the last dict

        workout_listdict.append(workout_dict)
    
    return workout_listdict

# Extract

In [22]:
raw_workouts = get_raw_workouts(r'G:\My Drive\Projetos\workouts\2024_08_25 Workouts.csv')
raw_workouts.reverse()

# Transform

In [57]:
workouts_dict = get_workout_dict(raw_workouts)
pprint.pprint(workouts_dict[32], sort_dicts=False)

{'raw_description': '"Peito Ombro Triceps · Day 1 · Week 5 · Padrão3 · '
                    'Deload";"2024-01-05 11:58 h";"1:05 hr"',
 'description': {'day_name': 'Peito Ombro Triceps',
                 'day_number': 'Day 1',
                 'week_number': 'Week 5',
                 'plan_name': 'Padrão3',
                 'plan_function': 'Deload',
                 'start_time': '2024-01-05 11:58 h',
                 'duration': '1:05 hr'},
 'exercises': [{'exercise': '"1. Bench Press · Barbell · 12 reps"',
                'data': ['#;KG;REPS', '1;36;12', '2;36;12', '3;36;12']},
               {'exercise': '"2. Incline Bench Press · Barbell · 12 reps"',
                'data': ['#;KG;REPS', '1;24;12', '2;24;12', '3;24;12']},
               {'exercise': '"3. Seated Shoulder Press · Barbell · 12 reps"',
                'data': ['#;KG;REPS', '1;16;12', '2;16;12', '3;16;12']},
               {'exercise': '"4. Butterfly with Wide Grip · Machine · 12 reps"',
                'data': ['#;KG